In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re

from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister, Parameter

In [64]:
filename = 'yao_circuit'

def prepare_circuit_file(filename, rootpath='../data/'):
    nqubits = 0
    with open(rootpath + filename + '.txt', 'r') as r, open(rootpath + filename + '_clean.txt', 'w') as o:
        for line in r:
            isnotempty = line.strip()
            if 'nqubits' in line:
                nqubits = int(re.findall(r'\d+', line)[0])
            if isnotempty and ('nqubits' not in line) and ('chain' not in line):
                o.write(line)
    file = open(rootpath + filename + '_clean.txt', 'r').read()
    file = file.split('\n') # split file at breaklines
    file = file[:-1] # remove last stray empty line
    # file = [file[x:x+2] for x in range(0, len(file), 2)] # reshape instructions
    grouped_list = []
    current_group = []
    for line in file:
        if 'put on' in line or 'control' in line:
            if current_group:
                grouped_list.append(current_group)
            current_group = [line]
        else:
            current_group.append(line)
    if current_group:
        grouped_list.append(current_group)
    return grouped_list, nqubits

grouped_file, nqubits = prepare_circuit_file(filename)

In [65]:
def circ_translator(t, n, instructions, name='circ_from_yao'):
    q = QuantumRegister(n)
    global_angle = 0
    for instruction in instructions:
        if 'phase' in instruction[1]:
            global_angle += float(re.search(r'[-+]?(?:\d*\.*\d+)', instruction[1]).group())
    circ = QuantumCircuit(q, global_phase=global_angle*t, name=name)
    for instruction in instructions:
        command, *operands = instruction
        if 'put on' in command:
            qubit = int(re.search(r'\d+', command).group())-1
            for operand in operands:
                if 'X' in operand:
                    circ.x(qubit)
                if 'Y' in operand:
                    circ.y(qubit)
                if 'Z' in operand:
                    circ.z(qubit)
                if 'H' in operand:
                    circ.h(qubit)
                if 'shift' in operand:
                    angle = float(re.search(r'[-+]?(?:\d*\.*\d+)', operand).group())
                    circ.rz(angle, qubit)
                if 'rot' in operand:
                    gate = re.search('[A-Z]', operand).group()
                    angle = float(re.search(r'[-+]?(?:\d*\.*\d+)', operand).group())
                    if gate == 'X':
                        circ.rx(angle*t, qubit)
                    if gate == 'Y':
                        circ.ry(angle*t, qubit)
                    if gate == 'Z':
                        circ.rz(angle*t, qubit)
        elif 'control' in command:
            command_qubits = re.findall(r'\d+', command)
            control_qubit = int(command_qubits[0])-1
            target_qubits = []
            for i in range(1, len(command_qubits)):
                target_qubits.append(int(command_qubits[i])-1)
            for operand in operands:
                target_qubits.append(int(re.search(r'\d+', operand).group())-1)
            for operand in operands:
                gate = re.search('[A-Z]', operand).group()
                if gate in ['X', 'Y', 'Z']:
                    for target in target_qubits:
                        if gate == 'X':
                            circ.cx(control_qubit, target)
                        elif gate == 'Y':
                            circ.cy(control_qubit, target)
                        elif gate == 'Z':
                            circ.cz(control_qubit, target)
    return circ

# def circ_simplify(circ, keep_prm_uuid=True):
#     circ_tk = qiskit_to_tk(circ, keep_prm_uuid)
#     RemoveRedundancies().apply(circ_tk)
#     PauliSimp().apply(circ_tk)
#     # PauliSquash().apply(circ_tk)
#     # CliffordSimp().apply(circ)
#     # KAKDecomposition().apply(circ)
#     circ = tk_to_qiskit(circ_tk, keep_prm_uuid)
#     return circ

# t_prm = Parameter('t')

# use t_prm for time-dependent circuits (e.g. Cartan) otherwise use t=1
print(circ_translator(1, nqubits, grouped_file))

global phase: 0.3
       ┌───┐┌───┐      ┌───┐┌───┐      ┌───┐      
q44_0: ┤ H ├┤ X ├─■──■─┤ X ├┤ X ├─■──■─┤ X ├─■──■─
       ├───┤└───┘ │  │ ├───┤└───┘ │  │ ├───┤ │  │ 
q44_1: ┤ H ├──────■──┼─┤ X ├──────■──┼─┤ X ├─■──┼─
       ├───┤┌───┐    │ ├───┤         │ └───┘    │ 
q44_2: ┤ H ├┤ X ├────■─┤ X ├─────────■──────────■─
       └───┘└───┘      └───┘                      
